In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

#from Course.PyTorch_Intro.PyTorch_intro import vocab_size

# Hyperparameters
window_size = 100
seq_length = 99
embed_size = 128
hidden_size = 256
batch_size = 64
learning_rate = 0.001
num_epochs = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [3]:
# data preparation
def get_data():
    print("Text downloading...")
    url = "https://www.gutenberg.org/cache/epub/84/pg84-images.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.get_text()

    words = re.findall(r'\w+', text.lower())
    words = words[:50000]
    vocab = sorted(list(set(words)))
    word_to_ix = {word: i for i, word in enumerate(vocab)}
    ix_to_word = {i: word for i, word in enumerate(vocab)}

    print(f"Words count՝ {len(words)}")
    print(f"Unique words (Vocab size)՝ {len(vocab)}")

    inputs = []
    targets = []
    for i in range(len(words) - window_size + 1):
        input_seq = [word_to_ix[w] for w in words[i : i + seq_length]]
        target_word = word_to_ix[words[i + seq_length]]

        inputs.append(input_seq)
        targets.append(target_word)

    X = torch.LongTensor(inputs)
    y = torch.LongTensor(targets)

    return X, y, len(vocab), word_to_ix, ix_to_word

class TextGenRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super(TextGenRNN, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_size)

        # RNN Layer
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)

        # Fully Connected Layer
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        embeds = self.embedding(x) # (Batch_Size, 99, Embed_Size)
        out, hidden = self.rnn(embeds)
        last_output = out[:, -1, :]
        prediction = self.fc(last_output)

        return prediction

def generate_text(model, start_words, length, word_to_ix, ix_to_word):
    model.eval()

    current_words = start_words.split()
    input_seq = [word_to_ix.get(w, 0) for w in current_words]

    if len(input_seq) < seq_length:
        input_seq = [0] * (seq_length - len(input_seq)) + input_seq

    generated_text = list(current_words)

    with torch.no_grad():
        for _ in range(length):
            x = torch.LongTensor([input_seq[-seq_length:]]).to(device)

            # Forward pass
            output = model(x)

            probs = torch.softmax(output, dim=1).cpu().numpy()[0]


            next_ix = np.random.choice(len(probs), p=probs)
            next_word = ix_to_word[next_ix]

            generated_text.append(next_word)
            input_seq.append(next_ix)

    return " ".join(generated_text)

In [4]:
# Main
if __name__ == "__main__":
    X, y, vocab_size, word_to_ix, ix_to_word = get_data()
    dataset = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    model = TextGenRNN(vocab_size, embed_size, hidden_size).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # training loop
    print("Starting training...")
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch_inputs, batch_targets in dataloader:
            batch_inputs, batch_targets = batch_inputs.to(device), batch_targets.to(device)

            # Zero gradients
            optimizer.zero_grad()

            outputs = model(batch_inputs)
            # Loss Calculation
            loss = criterion(outputs, batch_targets)

            # Backward & Step
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

        sample_seed = "i saw the lightning playing on the summit of mont blanc"
        print(f"Sample: {generate_text(model, sample_seed, 10, word_to_ix, ix_to_word)}...")
        print("-" * 30)

    # Final Generation
    print("\n--- Final Training Results ---")
    seed_sentence = "the creature opened his dull yellow eye and looked at me with a grin"
    final_text = generate_text(model, seed_sentence, 100, word_to_ix, ix_to_word)
    print(final_text)

Text downloading...
Words count՝ 50000
Unique words (Vocab size)՝ 5873
Starting training...
Epoch 1/20, Loss: 6.3886
Sample: i saw the lightning playing on the summit of mont blanc by of evil brought he unfortunate the question might loose...
------------------------------
Epoch 2/20, Loss: 5.4749
Sample: i saw the lightning playing on the summit of mont blanc or the bid you and only and he diffused skirting...
------------------------------
Epoch 3/20, Loss: 4.8217
Sample: i saw the lightning playing on the summit of mont blanc with fury break our cares with delight bread within me...
------------------------------
Epoch 4/20, Loss: 4.1979
Sample: i saw the lightning playing on the summit of mont blanc me seated mine they were every one else food when...
------------------------------
Epoch 5/20, Loss: 3.6121
Sample: i saw the lightning playing on the summit of mont blanc as i spoke in a quick air that he said...
------------------------------
Epoch 6/20, Loss: 3.0877
Sample: i saw th